In [1]:
import svgwrite

def render_card_svg(card, filename):
    dwg = svgwrite.Drawing(filename, size=("100px", "150px"))

    shape_map = {
        "diamond": lambda x, y: dwg.polygon([(x, y), (x+40, y+60), (x-40, y+60)]),
        "oval": lambda x, y: dwg.ellipse(center=(x, y), r=(20, 40)),
        "squiggle": lambda x, y: dwg.path(d=f"M{x},{y} C{x+20},{y+30} {x-20},{y+60} {x},{y+90}")
    }

    for i in range(card.number):
        x = 50
        y = 30 + i*40
        shape = shape_map[card.shape](x, y)

        if card.shading == "solid":
            shape.fill(color=card.color)
        elif card.shading == "striped":
            shape.fill("none").stroke(card.color, width=2)
            # optionally add lines for stripes
        elif card.shading == "open":
            shape.fill("none").stroke(card.color, width=2)

        dwg.add(shape)

    dwg.save()


In [7]:
import os
import svgwrite
from dataclasses import dataclass
from itertools import product

# --------------------------
# Define Card Dataclass
# --------------------------
@dataclass(frozen=True)
class Card:
    color: str
    shape: str
    number: int
    shading: str

# --------------------------
# Feature Options
# --------------------------
colors = ["red", "green", "purple"]
shapes = ["diamond", "oval", "squiggle"]
numbers = [1, 2, 3]
shadings = ["solid", "striped", "open"]

# --------------------------
# SVG Rendering Function
# --------------------------
def render_card_svg(card: Card, filename: str):
    dwg = svgwrite.Drawing(filename, size=("100px", "150px"))

    shape_map = {
        "diamond": lambda x, y: dwg.polygon([(x, y-20), (x+20, y), (x, y+20), (x-20, y)]),
        "oval": lambda x, y: dwg.ellipse(center=(x, y), r=(40, 20)),
        "squiggle": lambda x, y: dwg.path(d=f"M{x},{y} C{x+15},{y+20} {x-15},{y+40} {x},{y+60}")
    }

    for i in range(card.number):
        x = 50
        y = 30 + i*40
        shape = shape_map[card.shape](x, y)

        if card.shading == "solid":
            shape.fill(color=card.color)
        elif card.shading == "striped":
            shape.fill("none").stroke(card.color, width=2)
            # optional: add pattern for stripes here
        elif card.shading == "open":
            shape.fill("none").stroke(card.color, width=2)

        dwg.add(shape)

    dwg.save()

# --------------------------
# Main Script
# --------------------------
def main(output_folder="cards"):
    os.makedirs(output_folder, exist_ok=True)

    all_cards = [
        Card(color, shape, number, shading)
        for color, shape, number, shading in product(colors, shapes, numbers, shadings)
    ]

    for card in all_cards:
        # generate a filename like red_diamond_1_solid.svg
        filename = f"{card.color}_{card.shape}_{card.number}_{card.shading}.svg"
        filepath = os.path.join(output_folder, filename)
        render_card_svg(card, filepath)

    print(f"Generated {len(all_cards)} cards in '{output_folder}'")

# --------------------------
if __name__ == "__main__":
    main()


Generated 81 cards in 'cards'


In [9]:
import os
import svgwrite
from dataclasses import dataclass
from itertools import product
import cairosvg

# --------------------------
# Card Dataclass
# --------------------------
@dataclass(frozen=True)
class Card:
    color: str
    shape: str
    number: int
    shading: str

colors = ["red", "green", "purple"]
shapes = ["diamond", "oval", "squiggle"]
numbers = [1, 2, 3]
shadings = ["solid", "striped", "open"]

# --------------------------
# Helper: Draw Striped Pattern
# --------------------------
def add_stripes(dwg, x, y, width=20, height=40, spacing=5, color="black"):
    pattern = dwg.pattern(id="stripes", patternUnits="userSpaceOnUse", size=(spacing*2, spacing*2))
    # simple diagonal lines
    pattern.add(dwg.line(start=(0,0), end=(spacing*2, spacing*2), stroke=color, stroke_width=1))
    dwg.defs.add(pattern)
    rect = dwg.rect(insert=(x-10, y-20), size=(width, height), fill="url(#stripes)")
    dwg.add(rect)

# --------------------------
# Render Card SVG
# --------------------------
def render_card_svg(card: Card, filename: str):
    dwg = svgwrite.Drawing(filename, size=("100px", "150px"))

    for i in range(card.number):
        y = 30 + i*40
        shape_elem = None

        # Shapes
        if card.shape == "diamond":
            shape_elem = dwg.polygon([(50, y-20), (70, y), (50, y+20), (30, y)])
        elif card.shape == "oval":
            shape_elem = dwg.ellipse(center=(50, y), r=(40, 20))
        elif card.shape == "squiggle":
            shape_elem = dwg.path(d=f"M50,{y} C65,{y+20} 35,{y+40} 50,{y+60}")

        # Fill / stroke
        if card.shading == "solid":
            shape_elem.fill(color=card.color)
            shape_elem.stroke(card.color)
            dwg.add(shape_elem)
        elif card.shading == "striped":
            shape_elem.fill("none")
            shape_elem.stroke(card.color, width=2)
            dwg.add(shape_elem)
            # add stripes as lines inside the shape bounding box
            for line_y in range(int(y-20), int(y+20), 4):
                dwg.add(dwg.line(start=(30,line_y), end=(70,line_y), stroke=card.color, stroke_width=1))
        elif card.shading == "open":
            shape_elem.fill("none")
            shape_elem.stroke(card.color, width=2)
            dwg.add(shape_elem)

    dwg.save()

    # Optional: also export PNG
    cairosvg.svg2png(url=filename, write_to=filename.replace(".svg", ".png"))

# --------------------------
# Generate All Cards
# --------------------------
def main(output_folder="cards"):
    os.makedirs(output_folder, exist_ok=True)

    all_cards = [
        Card(color, shape, number, shading)
        for color, shape, number, shading in product(colors, shapes, numbers, shadings)
    ]

    for card in all_cards:
        filename = f"{card.color}_{card.shape}_{card.number}_{card.shading}.svg"
        filepath = os.path.join(output_folder, filename)
        render_card_svg(card, filepath)

    print(f"Generated {len(all_cards)} cards in '{output_folder}' (SVG + PNG)")

if __name__ == "__main__":
    main()


Generated 81 cards in 'cards' (SVG + PNG)
